# Lazy vs Eager

Understand the difference between eager and lazy compilation and transpilation.

⚠️ If you are running this notebook in Colab, you will have to install `Ivy` and some dependencies manually. You can do so by running the cell below ⬇️

If you want to run the notebook locally but don't have Ivy installed just yet, you can check out the [Get Started section of the docs.](https://unify.ai/docs/ivy/overview/get_started.html)

In [ ]:
!pip install ivy

`ivy.unify`, `ivy.compile` and `ivy.transpile` can all be performed either eagerly or lazily. All previous examples have been performed **lazily**, which means that the unification, compilation, or transpilation process actually occurs during the first call of the **returned** function. 

This is because all three of these processes depend on function tracing, which requires function arguments to use for the tracing. Alternatively, the arguments can be provided during the `ivy.unify`, `ivy.compile` or `ivy.transpile` call itself, in which case the process is performed **eagerly**. We show some simple examples for each case below.

## Unify

Consider again this simple `torch` function:

In [2]:
import ivy
import torch

def normalize(x):
    mean = torch.mean(x)
    std = torch.std(x)
    return torch.div(torch.sub(x, mean), std)

And let's also create the dummy `numpy` arrays as before:

In [4]:
# import NumPy
import numpy as np

# create random numpy array for testing
x = np.random.uniform(size=10)

Let's assume that our target framework is `tensorflow`:

In [6]:
import tensorflow as tf
ivy.set_backend("tensorflow")

x = tf.constant(x)

In the example below, the function is unified **lazily**, which means the first function call will execute slowly, as this is when the unification process actually occurs.

In [7]:
norm = ivy.unify(normalize, source="torch")
norm(x) # slow, lazy unification
norm(x) # fast, unified on previous call

ivy.array([-0.54320029,  1.30825614,  1.17176882,  1.14351968, -0.98934778,
        0.82910388, -0.89044143, -0.71881472, -0.1666683 , -1.14417601])

However, in the following example the unification occurs **eagerly**, and both function calls will be fast:

In [8]:
ivy.set_backend("tensorflow")
norm = ivy.unify(normalize, source="torch", args=(x,))
norm(x) # fast, unified at ivy.unify
norm(x) # fast, unified at ivy.unify

ivy.array([-0.54320029,  1.30825614,  1.17176882,  1.14351968, -0.98934778,
        0.82910388, -0.89044143, -0.71881472, -0.1666683 , -1.14417601])

## Compile

The same is true for compiling. In the example below, the function is compiled **lazily**, which means the first function call will execute slowly, as this is when the compilation process actually occurs.

In [9]:
norm_comp = ivy.compile(norm)
norm_comp(x) # slow, lazy compilation
norm_comp(x) # fast, compiled on previous call

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.54320029,  1.30825614,  1.17176882,  1.14351968, -0.98934778,
        0.82910388, -0.89044143, -0.71881472, -0.1666683 , -1.14417601])>

However, in the following example the compilation occurs **eagerly**, and both function calls will be fast:

In [10]:
norm_comp = ivy.compile(norm, args=(x,))
norm_comp(x) # fast, compiled at ivy.compile
norm_comp(x) # fast, compiled at ivy.compile

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.54320029,  1.30825614,  1.17176882,  1.14351968, -0.98934778,
        0.82910388, -0.89044143, -0.71881472, -0.1666683 , -1.14417601])>

## Transpile

The same is true for transpiling. In the example below, the function is transpiled **lazily**, which means the first function call will execute slowly, as this is when the transpilation process actually occurs.

In [11]:
norm_trans = ivy.transpile(normalize, source="torch", to="tensorflow")
norm_trans(x) # slow, lazy transpilation
norm_trans(x) # fast, transpiled on previous call

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.54320029,  1.30825614,  1.17176882,  1.14351968, -0.98934778,
        0.82910388, -0.89044143, -0.71881472, -0.1666683 , -1.14417601])>

However, in the following example the transpilation occurs *eagerly*, and both function calls will be fast:

In [12]:
norm_trans = ivy.transpile(normalize, source="torch", to="tensorflow", args=(x,))
norm_trans(x) # fast, transpiled at ivy.transpile
norm_trans(x) # fast, transpiled at ivy.transpile

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.54320029,  1.30825614,  1.17176882,  1.14351968, -0.98934778,
        0.82910388, -0.89044143, -0.71881472, -0.1666683 , -1.14417601])>

## Round Up

That's it, you now know the difference between lazy vs eager execution for `ivy.unify`, `ivy.compile` and `ivy.transpile`! Next, we'll be exploring how these three functions can all be called as function decorators!